In [2]:
import pandas as pd

In [35]:
import pandas as pd
import numpy as np
import json
import re 
import sys
import itertools

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler


import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import spotipy.util as util

import warnings
warnings.filterwarnings("ignore")

In [14]:
movie_df = pd.read_csv('movie_df.csv')
print(movie_df)

                             movie            processed_movie  \
0        Marvel Cinematic Universe  marvel cinematic universe   
1                        Star Wars                  star wars   
2         Harry Potter (franchise)               harry potter   
3             The Lion King (1994)              the lion king   
4                Shrek (Franchise)                      shrek   
...                            ...                        ...   
2776      The incredible Hulk 2007   the incredible hulk 2007   
2777                   The Killers                the killers   
2778            Logan's Run (1976)                logan's run   
2779            The 6th Day (2000)                the 6th day   
2780             The Switch (2010)                 the switch   

                   expanded-genres  \
0                              NaN   
1       Action, Adventure, Fantasy   
2                              NaN   
3      Animation, Adventure, Drama   
4     Animation, Adventure, C

In [15]:
movie_df.drop('expanded-genres', axis=1, inplace=True)


In [21]:
movie_df.to_csv('best.csv', index=False)

In [22]:
best = pd.read_csv('best.csv')

In [23]:
print(best)
print(best.dtypes)


                             movie            processed_movie  \
0        Marvel Cinematic Universe  marvel cinematic universe   
1                        Star Wars                  star wars   
2         Harry Potter (franchise)               harry potter   
3             The Lion King (1994)              the lion king   
4                Shrek (Franchise)                      shrek   
...                            ...                        ...   
2417      The incredible Hulk 2007   the incredible hulk 2007   
2418                   The Killers                the killers   
2419            Logan's Run (1976)                logan's run   
2420            The 6th Day (2000)                the 6th day   
2421             The Switch (2010)                 the switch   

                                Associated Music Genres  
0                             alternative,pop,punk,jazz  
1     acoustic,folk,punk,ambient,dubstep,hard-rock,a...  
2                             alternative,pop

In [25]:
best = best.dropna(subset=['Associated Music Genres'])

# Count occurrences of each movie
nan_count = best['Associated Music Genres'].isna().sum()

print("Number of NaN values in the 'Associated Music Genres' column:", nan_count)

Number of NaN values in the 'Associated Music Genres' column: 0


In [ ]:
print(movie_df)

In [27]:
spotify_df = pd.read_csv('millionmusic.csv')

In [28]:
spotify_df['popularity'].describe()

count    114000.000000
mean         33.238535
std          22.305078
min           0.000000
25%          17.000000
50%          35.000000
75%          50.000000
max         100.000000
Name: popularity, dtype: float64

In [29]:
spotify_df['popularity_red'] = spotify_df['popularity'].apply(lambda x: int(x/5))

In [32]:
na_count = spotify_df['track_genre'].isna().sum()

print("Number of NaN values in the 'track_genre' column:", na_count)
spotify_df.head()

Number of NaN values in the 'track_genre' column: 0


,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre,popularity_red
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,...,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic,14
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,...,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic,11
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,...,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic,11
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,...,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic,14
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,...,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic,16


In [33]:
#simple function to create OHE features
#this gets passed later on
def ohe_prep(df, column, new_name): 
    """ 
    Create One Hot Encoded features of a specific column

    Parameters: 
        df (pandas dataframe): Spotify Dataframe
        column (str): Column to be processed
        new_name (str): new column name to be used
        
    Returns: 
        tf_df: One hot encoded features 
    """
    
    tf_df = pd.get_dummies(df[column])
    feature_names = tf_df.columns
    tf_df.columns = [new_name + "|" + str(i) for i in feature_names]
    tf_df.reset_index(drop = True, inplace = True)    
    return tf_df

In [41]:
music_genre_metrics = pd.read_csv('genre_avg_metrics.csv')

In [61]:
import pandas as pd

best = pd.read_csv('best.csv')

music_genre_metrics = pd.read_csv('genre_avg_metrics.csv')

# Split genres in best dataframe
best['genres'] = best['Associated Music Genres'].str.split(',')

# Create an empty list to store merged dataframes
merged_dfs = []

# Perform individual merges for each genre
for index, row in best.iterrows():
    genres = row['genres']
    for genre in genres:
        genre_metrics = music_genre_metrics[music_genre_metrics['track_genre'] == genre]
        # Transpose the current row and reset index to ensure it's treated as a dataframe
        row_df = pd.DataFrame(row).T.reset_index(drop=True)
        # Add the 'track_genre' column to the transposed row dataframe
        row_df['track_genre'] = genre
        merged_df = pd.merge(row_df, genre_metrics, on='track_genre', how='left')
        merged_dfs.append(merged_df)

# Concatenate all resulting dataframes
final_merged_df = pd.concat(merged_dfs, ignore_index=True)

print(final_merged_df)


                              movie            processed_movie  \
0         Marvel Cinematic Universe  marvel cinematic universe   
1         Marvel Cinematic Universe  marvel cinematic universe   
2         Marvel Cinematic Universe  marvel cinematic universe   
3         Marvel Cinematic Universe  marvel cinematic universe   
4                         Star Wars                  star wars   
...                             ...                        ...   
32606             The Switch (2010)                 the switch   
32607             The Switch (2010)                 the switch   
32608             The Switch (2010)                 the switch   
32609             The Switch (2010)                 the switch   
32610             The Switch (2010)                 the switch   

                                 Associated Music Genres  \
0                              alternative,pop,punk,jazz   
1                              alternative,pop,punk,jazz   
2                          

In [66]:
genre_data = {}

# Iterate over each row
for index, row in final_merged_df.iterrows():
    movie = row['movie']
    genre = row['track_genre']
    
    # Create genre-specific column names
    for metric in ['danceability', 'energy', 'valence', 'loudness', 'tempo', 'speechiness']:
        column_name = f"{genre}_{metric}"
        # Add data to the dictionary
        if movie in genre_data:
            genre_data[movie][column_name] = row[metric]
        else:
            genre_data[movie] = {column_name: row[metric]}

# Create a DataFrame from the dictionary
genre_df = pd.DataFrame.from_dict(genre_data, orient='index')

print(genre_df)

                              alternative_danceability  alternative_energy  \
   Marvel Cinematic Universe                  0.559927             0.72003   
   Star Wars                                  0.559927             0.72003   
   Harry Potter (franchise)                   0.559927             0.72003   
   The Lion King (1994)                       0.559927             0.72003   
   Shrek (Franchise)                          0.559927             0.72003   
...                                                ...                 ...   
   A Scanner Darkly (2006)                         NaN                 NaN   
   Seven Psychopaths (2012)                        NaN                 NaN   
   Whisky Galore!                                  NaN                 NaN   
   Delicatessen (1991)                             NaN                 NaN   
   The Hustle (2019)                               NaN                 NaN   

                              alternative_valence  alternative_

In [67]:
genre_df_cleaned = genre_df.dropna(axis=1, how='all')

print(genre_df_cleaned)
print(genre_df_cleaned.columns)

                              alternative_danceability  alternative_energy  \
   Marvel Cinematic Universe                  0.559927             0.72003   
   Star Wars                                  0.559927             0.72003   
   Harry Potter (franchise)                   0.559927             0.72003   
   The Lion King (1994)                       0.559927             0.72003   
   Shrek (Franchise)                          0.559927             0.72003   
...                                                ...                 ...   
   A Scanner Darkly (2006)                         NaN                 NaN   
   Seven Psychopaths (2012)                        NaN                 NaN   
   Whisky Galore!                                  NaN                 NaN   
   Delicatessen (1991)                             NaN                 NaN   
   The Hustle (2019)                               NaN                 NaN   

                              alternative_valence  alternative_